In [1]:
!pip install -q kaggle
from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ifeanyinneji","key":"68b7b1b6c0d102f7d9941d0b3eff800a"}'}

In [2]:
# This cell creates a .kaggle hidden folder in our root directory and copies the kaggle.json file to the folder.
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [3]:
# This cell will download the dataset used for this project to the /content directory and also unzip the csv file.
!kaggle datasets download -d xcherry/games-of-all-time-from-metacritic
!unzip /content/games-of-all-time-from-metacritic.zip

  0% 0.00/2.42M [00:00<?, ?B/s]
100% 2.42M/2.42M [00:00<00:00, 136MB/s]
Archive:  /content/games-of-all-time-from-metacritic.zip
  inflating: games_of_all_time.csv   


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
import pandas as pd
df = pd.read_csv("games_of_all_time.csv")
df

,game_name,meta_score,user_score,platform,description,url,developer,genre,type,rating
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,['nintendo-64'],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E
1,Super Mario Galaxy,97.0,91.0,['wii'],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E
2,Super Mario Galaxy 2,97.0,91.0,['wii'],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer,E
3,Metroid Prime,97.0,89.0,['gamecube'],Samus returns in a new mission to unravel the ...,https://www.metacritic.com/game/gamecube/metro...,Retro Studios,"['Action', 'Shooter', 'First-Person', 'Sci-Fi']",singleplayer,T
4,Super Mario Odyssey,97.0,89.0,['switch'],New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E10+
...,...,...,...,...,...,...,...,...,...,...
8826,kill.switch,74.0,80.5,"['xbox', 'playstation-2']","In a world on the brink of global conflict, yo...",https://www.metacritic.com/game/xbox/killswitch,Namco,"['Action', 'Shooter', 'Third-Person', 'Modern']",singleplayer,T
8827,nail'd,69.0,76.0,['pc'],nail’d is all about eschewing boring realism f...,https://www.metacritic.com/game/pc/naild,Techland,"['Driving', 'Racing', 'Arcade', 'Automobile']",singleplayer,E10+
8828,oOo: Ascension,70.0,73.0,['xbox-one'],oOo: Ascension is a lightning-paced test of re...,https://www.metacritic.com/game/xbox-one/ooo-a...,Kenny Creanor,"['Action', 'General', 'Arcade']",multiplayer,E
8829,uDraw Studio,71.0,71.0,['wii'],"Packaged with the uDraw GameTablet, uDraw Stud...",https://www.metacritic.com/game/wii/udraw-studio,THQ,"['Miscellaneous', 'General']",NaN,E


In [ ]:
total_genres = []
for g in df["genre"]:
    g = g
    if g not in total_genres:
        total_genres.append(g)
total_genres

In [12]:
df.shape

(8831, 10)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8831 entries, 0 to 8830
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game_name    8831 non-null   object 
 1   meta_score   8831 non-null   float64
 2   user_score   8831 non-null   float64
 3   platform     8831 non-null   object 
 4   description  8831 non-null   object 
 5   url          8831 non-null   object 
 6   developer    8821 non-null   object 
 7   genre        8827 non-null   object 
 8   type         6727 non-null   object 
 9   rating       7691 non-null   object 
dtypes: float64(2), object(8)
memory usage: 690.0+ KB


In [14]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['genre'].values.astype('U'))

In [15]:
cos_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
indices=pd.Series(df.index,index=df['game_name'])
titles=df['game_name']
def recommend_game(title):
    idx = indices[title]
    similarity_scores = list(enumerate(cos_similarity[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]
    data = []
    for i in similarity_scores:
        item = []
        temp_df = df[df['game_name'] == indices.index[i[0]] ]
        item.extend(list(temp_df.drop_duplicates('game_name')['game_name'].values ))
        item.extend(list(temp_df.drop_duplicates('platform')['platform'].values ))
        item.extend(list(temp_df.drop_duplicates('game_name')['genre'].values ))
        item.extend(list(temp_df.drop_duplicates('game_name')['type'].values ))
        
        data.append(item)
    return data

In [28]:
recom_sys('Call of Duty')

[['Call of Duty Classic',
  "['playstation-3', 'xbox-360']",
  'https://www.metacritic.com/game/playstation-3/call-of-duty-classic',
  "['Action', 'Shooter', 'First-Person', 'Third-Person', 'Historic', 'Tactical', 'Arcade']",
  'multiplayer'],
 ['LawBreakers',
  "['playstation-4']",
  'https://www.metacritic.com/game/playstation-4/lawbreakers',
  "['Third-Person', 'Arcade', 'Action', 'Shooter', 'First-Person', 'Tactical']",
  'multiplayer'],
 ['Day of Defeat: Source',
  "['pc']",
  'https://www.metacritic.com/game/pc/day-of-defeat-source',
  "['Action', 'Shooter', 'First-Person', 'Historic', 'Tactical', 'Arcade']",
  'multiplayer'],
 ['Call of Duty: Black Ops',
  "['playstation-3', 'wii', 'ds']",
  'https://www.metacritic.com/game/playstation-3/call-of-duty-black-ops',
  "['Action', 'Shooter', 'First-Person', 'Historic', 'Tactical', 'Arcade']",
  'multiplayer'],
 ['Call of Duty: Black Ops - Escalation',
  "['xbox-360']",
  'https://www.metacritic.com/game/xbox-360/call-of-duty-black-op